In [2]:
#jaydeb sarker
#gy3312

##import packages
import csv
import numpy as np
import random
import math
import pandas as pd

#import from sklearn
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import random as sparse_random
from sklearn.random_projection import sparse_random_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import LabelEncoder,scale


In [3]:
def get_train_data():
    train_data = pd.read_csv("fashion-mnist_train.csv")
    return train_data

In [4]:
def get_test_data():
    test_data = pd.read_csv("fashion-mnist_test.csv")
    return test_data

In [5]:
train= get_train_data()
test=  get_test_data()

In [6]:
#X_features = train.iloc[:,1:23]
XTrain_features = train.iloc[:,1:785]
YTrain_features= train.iloc[:,0]

#taking test data
Xtest_features=test.iloc[:,1:785]
Ytest_features = test.iloc[:,0]

In [7]:
#test validation split
from sklearn.model_selection import train_test_split

X_train, X_Val, y_train, y_val = train_test_split(XTrain_features, YTrain_features, test_size=0.10, random_state=42)



In [8]:
#scaling the input train
scaler = StandardScaler()
XTrain_features = scaler.fit_transform(X_train)
X_val=StandardScaler().fit_transform(X_Val.values)

X_test = StandardScaler().fit_transform(Xtest_features.values)


In [9]:
from sklearn.decomposition import PCA

#use PCA to reduce the features
pca = PCA(n_components=10)

train_pca= pca.fit(XTrain_features).transform(XTrain_features)
val_pca=pca.fit(X_val).transform(X_val)

test_pca = pca.fit(X_test).transform(X_test)



In [10]:

class K_Means:
    def __init__(self, k, tol=0.001, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter

    def fit(self, data):

        self.centroids = {}
        #random centroid initialization
        for i in range(self.k):
            r = np.random.randint(0, 50)
            self.centroids[i] = data[r]

        for i in range(self.max_iter):
            self.classifications = {}

            for i in range(self.k):
                self.classifications[i] = []

            for featureset in data:
                distances = [np.linalg.norm(featureset-self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classifications[classification].append(featureset)

            prev_centroids = dict(self.centroids)

            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification],axis=0)

            optimized = True

            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/original_centroid*100.0) > self.tol:
                    #print(np.sum((current_centroid-original_centroid)/original_centroid*100.0))
                    optimized = False

            if optimized:
                break

    def predict(self, data):
        distances = [np.linalg.norm(data-self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification

In [11]:
import matplotlib.pyplot as plt

'''
number_of_clusters = 10
kmeans = K_Means(number_of_clusters)


kmeans.fit(train_pca)
kmeans.predict(test_pca)

colors = 10*["green","red","gold","blue","silver", "yellow", "brown", "violet", "purple"]

for centroid in kmeans.centroids:
    plt.scatter(kmeans.centroids[centroid][0], kmeans.centroids[centroid][1],
                marker="o", color="k", s=150, linewidths=5)

for classification in kmeans.classifications:
    color = colors[classification]
    for featureset in kmeans.classifications[classification]:
        plt.scatter(featureset[0], featureset[1], marker="x", color=color, s=150, linewidths=5)

plt.show()
'''

'\nnumber_of_clusters = 10\nkmeans = K_Means(number_of_clusters)\n\n\nkmeans.fit(train_pca)\nkmeans.predict(test_pca)\n\ncolors = 10*["green","red","gold","blue","silver", "yellow", "brown", "violet", "purple"]\n\nfor centroid in kmeans.centroids:\n    plt.scatter(kmeans.centroids[centroid][0], kmeans.centroids[centroid][1],\n                marker="o", color="k", s=150, linewidths=5)\n\nfor classification in kmeans.classifications:\n    color = colors[classification]\n    for featureset in kmeans.classifications[classification]:\n        plt.scatter(featureset[0], featureset[1], marker="x", color=color, s=150, linewidths=5)\n\nplt.show()\n'

In [12]:
from itertools import combinations
import math
from scipy.special import comb
def rand_index(labels_true, labels_pred):
    """given the true and predicted labels, 
    it will return the Rand Index.
    Args:
        labels_true: the real class labels 
        labels_pred:the predicted class labels
    Return:
        RAND_index
    """
    # create list of all combinations with the length of labels.
    my_pair = list(combinations(range(len(labels_true)), 2))

    def is_equal(x):
        return (x[0] == x[1])
    
    my_a = 0
    my_b = 0
    
    for i in range(len(my_pair)):
        if(is_equal((labels_true[my_pair[i][0]], labels_true[my_pair[i][1]])) == is_equal((labels_pred[my_pair[i][0]], labels_pred[my_pair[i][1]]))
           and is_equal((labels_pred[my_pair[i][0]], labels_pred[my_pair[i][1]])) == True):
            my_a += 1
        if(is_equal((labels_true[my_pair[i][0]], labels_true[my_pair[i][1]])) == is_equal((labels_pred[my_pair[i][0]], labels_pred[my_pair[i][1]]))
           and is_equal((labels_pred[my_pair[i][0]], labels_pred[my_pair[i][1]])) == False):
            my_b += 1
            
    #my_denom = math.comb(len(labels_true), 2)
    my_denom = comb(len(labels_true), 2)
    RAND_index = (my_a + my_b) / my_denom
    return RAND_index

In [13]:
y_val=y_val.values.tolist()


In [15]:
from sklearn.metrics.cluster import adjusted_rand_score
import numpy
bestK=0
rand_adj=0
for k in range(5,16):
    number_of_clusters = k
    kmeans = K_Means(k)
    
    y_pred=[]
    
    kmeans.fit(train_pca)
    
    for valp in val_pca:
        a=kmeans.predict(valp)
        y_pred.append(a)
    
    
    #y_pred=numpy.array(y_pred)
    rand_score=rand_index(y_val,y_pred)
    print("\nK value# ", k)
    print('\nRand score is: ',rand_score)
    #anindex=rand_score(y_val, y_pred)
    
    #rint('RandIndex',ranindex)
    
    scorerand=adjusted_rand_score(y_val, y_pred)
    print('\nAdj rand score is: ',scorerand)
    if(rand_adj<rand_score):
        rand_adj=rand_score
        bestK=k
    
print('Best K is', bestK)

/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



K value#  5

Rand score is:  0.5486427182308162

Adj rand score is:  0.05860319850683295

K value#  6

Rand score is:  0.8244069011501917

Adj rand score is:  0.2562503335035596


/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



K value#  7

Rand score is:  0.797223648385842

Adj rand score is:  0.21836345751366176

K value#  8

Rand score is:  0.8417599599933322

Adj rand score is:  0.26829803801905694


/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...


K value#  13

Rand score is:  0.8697107851308551

Adj rand score is:  0.3006251311317496


/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



K value#  14

Rand score is:  0.8712973273323331

Adj rand score is:  0.3014149539819299


/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



K value#  15

Rand score is:  0.8706984497416236

Adj rand score is:  0.29962341144825916
Best K is 14


In [16]:
#converting to list 
Ytest_features=Ytest_features.values.tolist()
for i in range(0,6):
    kmeans = K_Means(bestK)
    kmeans.fit(train_pca)
    y_pred=[]
    
    #kmeans.fit(train_pca)
    
    for valt in test_pca:
        a=kmeans.predict(valt)
        y_pred.append(a)
    
    rand_score=rand_index(Ytest_features,y_pred)
    print('\nRun# ', i)
    print('\nRand score is: ',rand_score)
    #anindex=rand_score(y_val, y_pred)
    
    #rint('RandIndex',ranindex)
    
    scorerand=adjusted_rand_score(Ytest_features, y_pred)
    print('Adj rand score is: ',scorerand)

/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Run#  0

Rand score is:  0.8874687868786879
Adj rand score is:  0.35979836971189566


/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Run#  1

Rand score is:  0.8791159515951595
Adj rand score is:  0.3310724225796255


/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...


Run#  3

Rand score is:  0.8843138313831383
Adj rand score is:  0.3526989700235944


/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Run#  4

Rand score is:  0.8788998699869986
Adj rand score is:  0.34984779280915973


/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/anaconda/envs/toxicity_detector/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Run#  5

Rand score is:  0.8872842084208421
Adj rand score is:  0.34459174361953115
